In [ ]:
] activate ..

In [1]:
using UntitledLanguage
import UntitledLanguage: create_lambda, val

[ Info: Precompiling UntitledLanguage [e96edc6f-9096-47bc-907c-7cc24c70b7c3]


In [2]:
tests = [
    """
    __begin__
        let fib(n)
           a: 1
           b: 0
           for _ in 0..n
              a, b: b, a+b
           return b
    """, 
    """
    set! foo (lambda (x) (+ x 3))
    """
]

2-element Vector{String}:
 "__begin__\n    let fib(n)\n      " ⋯ 47 bytes ⋯ "  a, b: b, a+b\n       return b\n"
 "set! foo (lambda (x) (+ x 3))\n"

In [3]:
function to_SExpr(t::Tuple)
    return SExpr(to_SExpr.(t))
end
function to_SExpr(s::Union{String, SubString})
    t = tryparse(Int64, s)
    if !(t isa Nothing)
        return Atom(t)
    end
    
    t = tryparse(Float64, s)
    if !(t isa Nothing)
        return Atom(t)
    end
    
    return Atom(Symbol(s))
end

to_SExpr (generic function with 2 methods)

In [4]:
function create_lambda(ast, env)
    body = ast.expr[3]
    if ast.expr[2] isa Atom
        name = val(ast.expr[2])
        return (args...) -> ast_walk(
            ast.expr[3],
            env = Environment(Dict{Symbol, Any}(name => args), env)
        )
    else
        names = val.(ast.expr[2].expr)
        return (args...) -> ast_walk(
            ast.expr[3], 
            env = Environment(env, names, args)
        )
    end
end

create_lambda (generic function with 1 method)

In [5]:
function split_comma(arr) # Not used, but for the future, for now only one item
    ret = []
    cur = []
    for a in arr
        if a != Symbol(",")
            push!(cur, a)
        else
            push!(ret, cur)
            cur = []
        end
    end
    push!(ret, cur)
    return [SExpr(tuple(r...)) for r in ret]
end

function split_comma(arr)
    @assert isodd(length(arr))
    @assert all(==(Atom(Symbol(","))).(arr[2:2:end]))
    return arr[1:2:end]
end
    

split_comma (generic function with 1 method)

In [6]:
BUILTINS = () -> Dict{Symbol, Any}(
    :__println__ => println,
    :add => (+),
    :mul => (*),
    :+ => (+),
    :* => (*),
    :is => (===),
    :(=) => (==),
    :> => (>),
    :< => (<), 
    :≥ => (≥), 
    :≤ => (≤),
    :list => (args...) -> SExpr(args),
    :__nl__ => Macro((args...) -> SExpr(args)),
    :__call__ => Macro((args...) -> se(args[1], split_comma(args[2:end])...)),
    :__tuple__ => Macro((args...) -> (println(args); SExpr(args))),
    :lambda_ => Macro((args...) -> SExpr((at(:lambda), SExpr(args[1].expr[2:end]), args[2]))), 
    :print => Macro((args...) -> se(:__println__, args...))
)

#7 (generic function with 1 method)

In [11]:
parser("""
        set! x 3
        set! y 4
        set! x mul(mul(x,mul(mul(x,x),x)),x)
        print add(x,y)
""")

("__nl__", "begin", ("__nl__", "set!", "x", "3"), ("__nl__", "set!", "y", "4"), ("__nl__", "set!", "x", ("__call__", "mul", ("__call__", "mul", "x", ",", ("__call__", "mul", ("__call__", "mul", "x", ",", "x"), ",", "x")), ",", "x")), ("__nl__", "print", ("__call__", "add", "x", ",", "y")))

In [14]:
ast_walk(to_SExpr(parser("""
        set! x 3
        set! y 4
        set! x mul(mul(x,mul(mul(x,x),x)),x)
        print add(x,y)
""")), env=Environment(BUILTINS()))

247


In [17]:
ast_walk(to_SExpr(parser("""
        set! f (lambda_ (x) add(x, 3))
        print f(3)
""")), env=Environment(BUILTINS()))

(Atom{Symbol}(:lambda_), SExpr((Atom{Symbol}(:__tuple__), Atom{Symbol}(:x))), SExpr((Atom{Symbol}(:__call__), Atom{Symbol}(:add), Atom{Symbol}(:x), Atom{Symbol}(Symbol(",")), Atom{Int64}(3))))
6


In [23]:
printstyled("asdf", '\n', color=:blue)

asdf


In [24]:
?printstyled

search: printstyled



```
printstyled([io], xs...; bold::Bool=false, underline::Bool=false, blink::Bool=false, reverse::Bool=false, hidden::Bool=false, color::Union{Symbol,Int}=:normal)
```

Print `xs` in a color specified as a symbol or integer, optionally in bold.

Keyword `color` may take any of the values `:normal`, `:default`, `:bold`, `:black`, `:blink`, `:blue`, `:cyan`, `:green`, `:hidden`, `:light_black`, `:light_blue`, `:light_cyan`, `:light_green`, `:light_magenta`, `:light_red`, `:light_white`, `:light_yellow`, `:magenta`, `:nothing`, `:red`, `:reverse`, `:underline`, `:white`, or  `:yellow` or an integer between 0 and 255 inclusive. Note that not all terminals support 256 colors.

Keywords `bold=true`, `underline=true`, `blink=true` are self-explanatory. Keyword `reverse=true` prints with foreground and background colors exchanged, and `hidden=true` should be invisibe in the terminal but can still be copied. These properties can be used in any combination.

See also [`print`](@ref), [`println`](@ref), [`show`](@ref).

!!! compat "Julia 1.7"
    Keywords except `color` and `bold` were added in Julia 1.7.



In [10]:
# ast_walk(
temp = to_SExpr(parser("""
    begin
        set! x 3
        set! y 4
        add(x,y)
        3
        """)).expr[end-2]#, env=Environment(BUILTINS()))

SExpr((Atom{Symbol}(:__nl__), Atom{Symbol}(:set!), Atom{Symbol}(:y), Atom{Int64}(4)))

In [11]:
BUILTINS()[:__nl__](temp.expr[2:end])

SExpr(((Atom{Symbol}(:set!), Atom{Symbol}(:y), Atom{Int64}(4)),))

In [12]:
function repl(env=Environment(BUILTINS()); prompt="logos> ")
    while true
        if prompt != ""
            print(prompt)
        end
        str = readline()
        if str == "exit"
            break
        end
        try
            ast_walk(to_SExpr(parser(str)), env=env)
        catch e
            println(e)
        end
    end
end

repl (generic function with 2 methods)

In [13]:
repl(prompt="")

stdin>  set! x 2
stdin>  print x


2


stdin>  exit()


AssertionError("isodd(length(arr))")


stdin>  exit


In [25]:
a = """
let fib(n)
   a: 1
   b: 0
   for _ in 0..n
      a, b: b, a+b
   return b
"""

"let fib(n)\n   a: 1\n   b: 0\n   for _ in 0..n\n      a, b: b, a+b\n   return b\n"

In [27]:
.("    ", split(a, '\n'), '\n')

    let fib(n)
       a: 1
       b: 0
       for _ in 0..n
          a, b: b, a+b
       return b
    


7-element Vector{Nothing}:
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing